In [88]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
import re
from re import sub
from decimal import Decimal
import random
import pandas as pd
from urllib.request import urlopen
import dateutil.parser
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 12]


from fake_useragent import UserAgent
import os



chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver





In [89]:
#setting up fake user agent
ua = UserAgent()
user_agent = {'User-agent': ua.random}

#setting user agent names
opts=Options()
opts.add_argument('user-agent=user_agent')


In [90]:
driver = webdriver.Chrome(chromedriver, chrome_options=opts)
#only the recently sold homes
driver.get(url="https://www.zillow.com/ca/sold/")

In [91]:
#enter zipcode
#I will only have 500 homes for each zipcode allocated around all places because
#zillow requires you to zoom in to different map areas for more houses
zipcode = driver.find_element_by_xpath("//input[@id='citystatezip']")
zipcode.send_keys(Keys.BACKSPACE)
zipcode.send_keys(Keys.BACKSPACE)
zipcode.send_keys("95117")
zipcode.send_keys(Keys.RETURN)

In [92]:
#click on next page
driver.find_element_by_class_name('zsg-pagination-next').click()


In [93]:
#second page url
url2=driver.current_url
url2

'https://www.zillow.com/homes/recently_sold/San-Jose-CA-95117/97985_rid/globalrelevanceex_sort/37.343208,-121.908503,37.28095,-122.0218_rect/12_zm/2_p/'

In [94]:
##### get the url to feed into beautifulsoup
url_list=[]
url1=url2[:-4]+str(1)+url2[-3:]
url_list.append(url1)
url_list.append(url2)
#for i in range (page 3 to total page+1)
for i in range(3,21):
    url_list.append(url2[:-4]+str(i)+url2[-3:])
url_list

['https://www.zillow.com/homes/recently_sold/San-Jose-CA-95117/97985_rid/globalrelevanceex_sort/37.343208,-121.908503,37.28095,-122.0218_rect/12_zm/1_p/',
 'https://www.zillow.com/homes/recently_sold/San-Jose-CA-95117/97985_rid/globalrelevanceex_sort/37.343208,-121.908503,37.28095,-122.0218_rect/12_zm/2_p/',
 'https://www.zillow.com/homes/recently_sold/San-Jose-CA-95117/97985_rid/globalrelevanceex_sort/37.343208,-121.908503,37.28095,-122.0218_rect/12_zm/3_p/',
 'https://www.zillow.com/homes/recently_sold/San-Jose-CA-95117/97985_rid/globalrelevanceex_sort/37.343208,-121.908503,37.28095,-122.0218_rect/12_zm/4_p/',
 'https://www.zillow.com/homes/recently_sold/San-Jose-CA-95117/97985_rid/globalrelevanceex_sort/37.343208,-121.908503,37.28095,-122.0218_rect/12_zm/5_p/',
 'https://www.zillow.com/homes/recently_sold/San-Jose-CA-95117/97985_rid/globalrelevanceex_sort/37.343208,-121.908503,37.28095,-122.0218_rect/12_zm/6_p/',
 'https://www.zillow.com/homes/recently_sold/San-Jose-CA-95117/97985_r

In [95]:
#loop through each url in the url_list to get the house url data
house_url=[]
for i in url_list:
    #random user-agent
    ua = UserAgent()
    user_agent = {'User-agent': ua.random}
    #start using beautiful soup
    response = requests.get(i, headers=user_agent)
    sleeptime=random.uniform(3,5)
    time.sleep(sleeptime)
    print(response.status_code)
    soup=BeautifulSoup(response.text, "lxml")
    #url for individual houses
    for i in soup.find_all("a", {"class" :"zsg-photo-card-overlay-link routable hdp-link routable mask hdp-link"}):
        house_url.append('zillow.com'+i['href'])


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [96]:
house_url

['zillow.com/homedetails/998-Daffodil-Way-San-Jose-CA-95117/19605438_zpid/',
 'zillow.com/homedetails/519-Northlake-Dr-APT-2-San-Jose-CA-95117/2100850981_zpid/',
 'zillow.com/homedetails/4034-Payne-Ave-San-Jose-CA-95117/19612085_zpid/',
 'zillow.com/homedetails/1255-Flora-Ave-San-Jose-CA-95117/19604261_zpid/',
 'zillow.com/homedetails/1359-Phelps-Ave-APT-8-San-Jose-CA-95117/19609319_zpid/',
 'zillow.com/homedetails/3564-Parkland-Ave-San-Jose-CA-95117/19604615_zpid/',
 'zillow.com/homedetails/556-Westridge-Dr-San-Jose-CA-95117/19607458_zpid/',
 'zillow.com/homedetails/626-Coakley-Dr-San-Jose-CA-95117/19605839_zpid/',
 'zillow.com/homedetails/3128-Loma-Verde-Dr-APT-118-San-Jose-CA-95117/82952170_zpid/',
 'zillow.com/homedetails/411-Northlake-Dr-APT-5-San-Jose-CA-95117/19608550_zpid/',
 'zillow.com/homedetails/1411-Pinehurst-Sq-San-Jose-CA-95117/19609126_zpid/',
 'zillow.com/homedetails/785-Pineview-Dr-San-Jose-CA-95117/19607415_zpid/',
 'zillow.com/homedetails/458-Holly-Hock-Ct-San-Jose-

In [97]:
#save house_url into a csv file
df = pd.DataFrame(house_url)
df.to_csv('house_url_95117.csv', index=False)

In [98]:
#import the csv back to a list
import csv
with open('house_url_95117.csv', 'r') as f:
    reader = csv.reader(f)
    house_url = list(reader)
    
house_url.remove(house_url[0])

print(len(house_url))


500


In [99]:
#delist url
house_url = [y for x in house_url for y in x]


In [100]:
len(house_url)

500

## Open individual houses to get the characteristics using the house_url csv file stored from selenium

In [101]:
#open individual house's url using beautiful soup and extract all housing info

#setting up dataframe as a list
df_columns=['street address','city','state','zip',
            'bed','bath','sqft','type','year-built',
            'heating','cooling','parking','lot',
           'school1','school2','school3','grade1','grade2','grade3','rate1','rate2','rate3',
           'sold-price','sold-date','url']
data=[]

for j in house_url:
    #using random user-agent
    ua = UserAgent()
    user_agent = {'User-agent': ua.random}
    #feed into beautiful soup
    response = requests.get(url='https://www.'+j, headers=user_agent)
    sleeptime=random.uniform(3,5)
    time.sleep(sleeptime)
    print(response.status_code)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    #getting individual house characteristics
    housedata=[]
    #street address
    for i in soup.find_all("header", {"class" : "zsg-content-header addr"}):
        housedata.append(i.text.split(',')[0])
    #city, state, zip
    for i in soup.find_all("span", {"class" : "zsg-h2 addr_city"}):
        city=i.text.split(',')
        housedata.append(city[0])
        housedata.append(city[1].split(' ')[1])
        housedata.append(city[1].split(' ')[2])
    #bed, bath, sqft
    for i in soup.find_all("span", {"class" : "addr_bbs"}):
        housedata.append(i.text.split(' ')[0])
    #facts and features table: type, year built, heating, cooling, parking, lotsize
    for i in soup.find_all("div", {"class" :"hdp-fact-ataglance-value"}):
        housedata.append(i.text.strip())
#     #remodel year
#     for i in soup.find_all("span", {"class" : "hdp-fact-value"}):
#         if 'Built in' in i.text:
#             builtin=i.text.split(' ')[2]
#         if 'remodel year' in i.text:
#             remodel=i.text.split(':')[1]
#             print(remodel)
#         else: 
#             remodel=builtin
#     housedata.append(remodel)
    #school name
    for i in soup.find_all("div", {"class" :'nearby-schools-name'}):
        housedata.append(i.text.strip())    
    #school grades
    for i in soup.find_all("div", {"class" :'nearby-schools-grades'}):
        housedata.append(i.text.strip())
    #school rating 
    for i in soup.find_all("div", {"class" :'nearby-schools-rating'}):
        housedata.append(i.text.strip().split(' ')[0])
    
    #prices
    
    price=[]
    for i in soup.find_all("div", {"class" : "home-summary-row"}):
        #housedata.append(i.text.strip())
        price.append(i.text.strip())

    try:
        sold_price=Decimal(sub(r'[^\d.]','',re.findall('[£$€]{1}[,0-9]{1,10}',price[0])[0]))
        sold_date=dateutil.parser.parse(price[1],fuzzy=True)
    except IndexError:
        print("for sale houses")
        continue
        
    housedata.append(sold_price)
    housedata.append(sold_date)

        
    housedata.append('https://www.'+j)

    #append list of individual house characteristics into data
    data.append(housedata)

    


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
for sale houses
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [102]:
#converting the data list into a dataframe
housedf=pd.DataFrame(data,columns=df_columns)

In [103]:
housedf

,street address,city,state,zip,bed,bath,sqft,type,year-built,heating,cooling,parking,lot,shool1,school2,school3,grade1,grade2,grade3,rate1,rate2,rate3,sold-price,sold-date,url
0,998 Daffodil Way,San Jose,CA,95117,3,2,"1,250",Single Family,1962,Forced air,None,2 spaces,"6,634 sqft",Easterbrook Discovery,Prospect High,George C. Payne Elementary,K-8,9-12,K-5,9,9,9,1090000,2017-07-10,https://www.zillow.com/homedetails/998-Daffodi...
1,519 Northlake Dr APT 2,San Jose,CA,95117,2,1,800,Apartment,No Data,No Data,No Data,"Carport, Off street",No Data,Lynhaven Elementary,Monroe Middle,Del Mar High,K-5,5-8,9-12,6,5,7,1630000,2017-06-30,https://www.zillow.com/homedetails/519-Northla...
2,4034 Payne Ave,San Jose,CA,95117,3,2,"1,248",Single Family,1958,Other,No Data,2 spaces,"6,098 sqft",George C. Payne Elementary,Prospect High,Easterbrook Discovery,K-5,9-12,K-8,9,9,9,1200006,2017-06-23,https://www.zillow.com/homedetails/4034-Payne-...
3,1255 Flora Ave,San Jose,CA,95117,5,2,"2,074",Multi Family,1960,No Data,No Data,"Garage - Attached, On street",No Data,Leroy Anderson Elementary,Prospect High,Easterbrook Discovery,K-5,9-12,K-8,3,9,9,1170000,2017-06-19,https://www.zillow.com/homedetails/1255-Flora-...
4,1359 Phelps Ave APT 8,San Jose,CA,95117,3,2,"1,107",Condo,1963,Other,No Data,1 space,$287/month,George C. Payne Elementary,Easterbrook Discovery,Boynton High,K-5,K-8,9-12,9,9,NR,605000,2017-06-15,https://www.zillow.com/homedetails/1359-Phelps...
5,3564 Parkland Ave,San Jose,CA,95117,5,3,"2,199",Single Family,1962,Forced air,Central,2 spaces,"6,098 sqft",George C. Payne Elementary,Prospect High,Campbell Middle,K-5,9-12,5-8,9,9,3,1450000,2017-06-14,https://www.zillow.com/homedetails/3564-Parkla...
6,556 Westridge Dr,San Jose,CA,95117,3,2,"1,291",Single Family,1955,Other,No Data,2 spaces,"6,054 sqft",Merritt Trace Elementary,Herbert Hoover Middle,Abraham Lincoln High,K-5,6-8,9-12,6,4,7,1200000,2017-06-09,https://www.zillow.com/homedetails/556-Westrid...
7,626 Coakley Dr,San Jose,CA,95117,3,2,"1,472",Single Family,1955,Other,No Data,2 spaces,"6,838 sqft",Lynhaven Elementary,Monroe Middle,Del Mar High,K-5,5-8,9-12,6,5,7,1058000,2017-06-09,https://www.zillow.com/homedetails/626-Coakley...
8,3128 Loma Verde Dr APT 118,San Jose,CA,95117,2,2,932,Condo,2007,Forced air,"Central, Other","Carport, Off street, On street",$285/month,Rosemary Elementary,Campbell Middle,Westmont High,K-4,5-8,9-12,4,3,9,605000,2017-06-09,https://www.zillow.com/homedetails/3128-Loma-V...
9,411 Northlake Dr APT 5,San Jose,CA,95117,3,3,"1,626",Condo,1985,Other,No Data,2 spaces,$458/month,Lynhaven Elementary,Monroe Middle,Del Mar High,K-5,5-8,9-12,6,5,7,848000,2017-06-07,https://www.zillow.com/homedetails/411-Northla...


In [104]:
#save data into a csv file
housedf.to_csv('data_95117.csv', index=False)
#studio will have 0 bedrooms

## individual house testing case

In [514]:
# ua = UserAgent()
# user_agent = {'User-agent': ua.random}
# response = requests.get(url='https://www.zillow.com/homes/recently_sold/Atherton-CA/15577771_zpid/30280_rid/globalrelevanceex_sort/37.48705,-122.131662,37.413459,-122.268991_rect/12_zm/', headers=user_agent)
# page = response.text
# soup = BeautifulSoup(page,"lxml")
# print(soup.prettify())


In [ ]:
#remodel year part, couldn't get the correct data
# # ua = UserAgent()
# # user_agent = {'User-agent': ua.random}
# # #feed into beautiful soup
# # response = requests.get(url='https://www.zillow.com/homes/recently_sold/19596003_zpid/globalrelevanceex_sort/37.378137,-121.895972,37.304439,-122.033301_rect/12_zm/', headers=user_agent)
# # sleeptime=random.uniform(3,5)
# # time.sleep(sleeptime)
# # print(response.status_code)
# # page = response.text
# # soup = BeautifulSoup(page,"lxml")
# house=[]
# for i in soup.find_all("span", {"class" : "hdp-fact-name"}):
#     print(i.text)
# #   if 'Built in' in i.text:
# #         remodel=i.text.split(' ')[2]
# #     if 'remodel year' in i.text:
# #         print(i.text)
# #         #remodel=i.text.split(':')[1]
    
# # house.append(remodel)
# # house

In [236]:
'https://www.'+house_url[1]

'https://www.zillow.com/homedetails/1828-Clay-St-Santa-Clara-CA-95050/19579284_zpid/'

In [532]:
df_columns=['street address','city','state','zip',
            'bed','bath','sqft','type','year-built',
            'heating','cooling','parking','lot','year-remodel',
           'shool1','school2','school3','grade1','grade2','grade3','rate1','rate2','rate3',
           'sold-price','sold-date','zestimate','url']



data=[]



In [533]:
housedata=[]
#street address
for i in soup.find_all("span", {"class" : "zsg-breadcrumbs-text"}):
    street=i.text.strip()
    housedata.append(street)
#city, state, zip
for i in soup.find_all("span", {"class" : "zsg-h2 addr_city"}):
    city=i.text.strip().split(',')
    housedata.append(city[0])
    housedata.append(city[1].split(' ')[1])
    housedata.append(city[1].split(' ')[2])
#bed, bath, sqft
for i in soup.find_all("span", {"class" : "addr_bbs"}):
    housedata.append(i.text.split(' ')[0])
#facts and features table: type, year built, heating, cooling, parking, lotsize
for i in soup.find_all("div", {"class" :"hdp-fact-ataglance-value"}):
    housedata.append(i.text.strip())
#remodel year
for i in soup.find_all("span", {"class" : "hdp-fact-value"}):
    if 'Built in' in i.text:
        builtin=i.text.split(' ')[2]
    if 'remodel year' in i.text:
        remodel=i.text.split(':')[1]
    else: 
        remodel=builtin
housedata.append(remodel)
#school name
for i in soup.find_all("div", {"class" :'nearby-schools-name'}):
    housedata.append(i.text.strip())    
#school grades
for i in soup.find_all("div", {"class" :'nearby-schools-grades'}):
    housedata.append(i.text.strip())
#school rating 
for i in soup.find_all("div", {"class" :'nearby-schools-rating'}):
    housedata.append(i.text.strip().split(' ')[0])
#prices
prices=[]
for i in soup.find_all("div", {"class" : "home-summary-row"}):
    prices.append(i.text)

sold_price=Decimal(sub(r'[^\d.]','',re.findall('[£$€]{1}[,0-9]{1,10}',price[0])[0]))
sold_date=dateutil.parser.parse(price[1],fuzzy=True)
zestimate=Decimal(sub(r'[^\d.]','',re.findall('[£$€]{1}[,0-9]{1,10}',price[2])[0]))
housedata.append(sold_price)
housedata.append(sold_date)
housedata.append(zestimate)
housedata.append('https://www.'+house_url[0][0])


In [534]:
data.append(housedata)
data.append(housedata)



In [535]:
data

[['1828 Clay Street',
  'Santa Clara',
  'CA',
  '95050',
  '2',
  '2',
  '1,046',
  'Single Family',
  '1946',
  'Forced air',
  'None',
  'Garage - Detached',
  '4,791 sqft',
  '1946',
  'C. W. Haman Elementary',
  'Buchser Middle',
  'Santa Clara High',
  'K-5',
  '6-8',
  '9-12',
  '7',
  '6',
  '7',
  Decimal('1000000'),
  datetime.datetime(2017, 6, 26, 0, 0),
  Decimal('976416'),
  'https://www.zillow.com/homedetails/1873-Clifford-St-Santa-Clara-CA-95050/19579270_zpid/'],
 ['1828 Clay Street',
  'Santa Clara',
  'CA',
  '95050',
  '2',
  '2',
  '1,046',
  'Single Family',
  '1946',
  'Forced air',
  'None',
  'Garage - Detached',
  '4,791 sqft',
  '1946',
  'C. W. Haman Elementary',
  'Buchser Middle',
  'Santa Clara High',
  'K-5',
  '6-8',
  '9-12',
  '7',
  '6',
  '7',
  Decimal('1000000'),
  datetime.datetime(2017, 6, 26, 0, 0),
  Decimal('976416'),
  'https://www.zillow.com/homedetails/1873-Clifford-St-Santa-Clara-CA-95050/19579270_zpid/']]

In [536]:
pd.DataFrame(data,columns=df_columns)

,street address,city,state,zip,bed,bath,sqft,type,year-built,heating,cooling,parking,lot,year-remodel,shool1,school2,school3,grade1,grade2,grade3,rate1,rate2,rate3,sold-price,sold-date,zestimate,url
0,1828 Clay Street,Santa Clara,CA,95050,2,2,"1,046",Single Family,1946,Forced air,None,Garage - Detached,"4,791 sqft",1946,C. W. Haman Elementary,Buchser Middle,Santa Clara High,K-5,6-8,9-12,7,6,7,1000000,2017-06-26,976416,https://www.zillow.com/homedetails/1873-Cliffo...
1,1828 Clay Street,Santa Clara,CA,95050,2,2,"1,046",Single Family,1946,Forced air,None,Garage - Detached,"4,791 sqft",1946,C. W. Haman Elementary,Buchser Middle,Santa Clara High,K-5,6-8,9-12,7,6,7,1000000,2017-06-26,976416,https://www.zillow.com/homedetails/1873-Cliffo...


In [470]:
# # #street address
# # for i in soup.find_all("span", {"class" : "zsg-breadcrumbs-text"}):
# #     street=i.text.strip()
# #     housedata.append(street)
# #city, state, zip
# for i in soup.find_all("span", {"class" : "zsg-h2 addr_city"}):
#     city=i.text.strip().split(',')
# city[1].split(' ')[2]

'94027'

In [476]:
# #bed, bath, sqft
# for i in soup.find_all("span", {"class" : "addr_bbs"}):
#     print(i.text.split(' ')[0])
# bed

In [352]:
# #facts and features table: type, year built, heating, cooling, parking, lotsize
# for i in soup.find_all("div", {"class" :"hdp-fact-ataglance-value"}):
#     print(i.text.strip())


In [477]:
# # #school name
# # for i in soup.find_all("div", {"class" :'nearby-schools-name'}):
# #     housedata.append(i.text.strip())
    
# # #school name
# # for i in soup.find_all("div", {"class" :'nearby-schools-grades'}):
# #     housedata.append(i.text.strip())

# #school rating 
# schoolrate=[]
# for i in soup.find_all("div", {"class" :'nearby-schools-rating'}):
#     print(i.text.strip().split(' ')[0])
 

In [510]:
# features=[]
# #how to get part of the data, like year remodeled?
# for i in soup.find_all("span", {"class" : "hdp-fact-value"}):
#     if 'remodel' in i.text:
#         features.append(i.text)
#     else:
#         features.append('nan')

# features

In [511]:
# for i in soup.find_all("span", {"class" : "hdp-fact-value"}):
#     print(i.text)

In [394]:
# #prices
# prices=[]
# for i in soup.find_all("div", {"class" : "home-summary-row"}):
#     prices.append(i.text)

# sold_price=Decimal(sub(r'[^\d.]','',re.findall('[£$€]{1}[,0-9]{1,10}',price[0])[0]))
# sold_date=dateutil.parser.parse(price[1],fuzzy=True)
# zestimate=Decimal(sub(r'[^\d.]','',re.findall('[£$€]{1}[,0-9]{1,10}',price[2])[0]))



In [400]:
# check all span texts
# soup.find_all("span")


## Unused code

In [ ]:
'''
def get_html(driver):
    output = []
    keep_going = True
    while keep_going:
        # Pull page HTML
        try:
            output.append(driver.current_url)
        except TimeoutException:
            pass
        try:
            driver.find_element_by_class_name('zsg-pagination-next').click()
            time.sleep(3)
        except TimeoutException:
            keep_going = False
    return(output)
'''
#get_html(driver)

In [244]:
# #bedroom, bathroom, sqft
# for i in soup.find_all("span", {"class" : "zsg-photo-card-info"}):
#     print(i.text.strip())


# #street address
# for i in soup.find_all("span", {"itemprop" :"streetAddress"}):
#     print(i.text.strip())

# #city
# for i in soup.find_all("span", {"itemprop" :"addressLocality"}):
#     print(i.text.strip())

# #state
# for i in soup.find_all("span", {"itemprop" :"addressRegion"}):
#     print(i.text.strip())

# #zip code
# for i in soup.find_all("span", {"itemprop" :"postalCode"}):
#     print(i.text.strip())


# #url
# for i in soup.find_all("a", {"class" :"zsg-photo-card-overlay-link routable hdp-link routable mask hdp-link"}):
#     print('zillow.com'+i['href'])

# #within the specific house to get more data
# url1='https://www.zillow.com/homes/recently_sold/Santa-Clara-CA-95050/house_type/19579270_zpid/97951_rid/globalrelevanceex_sort/37.385503,-121.875544,37.312086,-122.024203_rect/12_zm/'
# response1 = requests.get(url1)
# response1.status_code
# page1 = response1.text
# soup1 = BeautifulSoup(page1,"lxml")
